In [0]:
%pip install psycopg2
%pip install azure-identity
%pip install azure-keyvault-secrets

In [0]:
dbutils.widgets.text("env", "dev", "Environment")
env = dbutils.widgets.get("env")

In [0]:

import json
import psycopg2
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
import constants

key_vault_name = constants.key_vault_name
dap_nonprod_sql_pwd_nm = constants.dap_nonprod_sql_pwd_nm

def get_secret(secret_name, key_vault_name):
    credential = DefaultAzureCredential()
    client = SecretClient(vault_url=f"https://{key_vault_name}.vault.azure.net/", credential=credential)
    return client.get_secret(secret_name).value

def get_db_connection():
    password = get_secret(dap_nonprod_sql_pwd_nm, key_vault_name)
    return psycopg2.connect(
        user=DB_CONFIG["user"],
        password=password,
        host=DB_CONFIG["host"],
        port=DB_CONFIG["port"],
        database=DB_CONFIG["database"]
    )

DB_CONFIG = {
    "user": constants.user,
    "host": constants.host,
    "port": constants.port,
    "database": constants.database.format(env = env),
}

with open("../regression/clear.json", "r") as file:
    confile_content = file.read()

queries = json.loads(confile_content)

conn = get_db_connection()
cur = conn.cursor()
for key in queries:
    cur.execute(queries[key])
    print(f"Executed {key} : {queries[key]}")
conn.commit()
cur.close()
conn.close()